In [8]:
import numpy as np
import pandas as pd #csv-files and tables
import keras #neural network

#create graphics
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

#utils for working with neural network
from sklearn.model_selection import train_test_split
from keras.utils import plot_model, to_categorical
from keras.layers import Dense
from keras.models import Sequential

#interactive in jupyter notebook
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [9]:
# activate widgets in terminal
# jupyter nbextension enable --py widgetsnbextension

In [14]:
#loading datasets
dataset_0 = pd.read_csv('data/simple.csv')
dataset_0.name = 'simple'
dataset_1 = pd.read_csv('data/iris_new.csv')
dataset_1.name = 'iris'
dataset_2 = pd.read_csv('data/cancer.csv')
dataset_2.name = 'cancer'
dataset_3 = pd.read_csv('data/glass.csv')
dataset_3.name = 'glass'
dataset_4 = pd.read_csv('data/thyroid.csv')
dataset_4.name = 'thyroid'
dataset_5 = pd.read_csv('data/vine.csv')
dataset_5.name = 'vine'

In [15]:
#data markup
#iloc -> select data on table

input_data_0 = dataset_0.iloc[:, :2]
output_data_0 = dataset_0.iloc[:, -4:]
io0 = [input_data_0.values, output_data_0.values]

input_data_1 = dataset_1.iloc[:, :4]
output_data_1 = dataset_1.iloc[:, -3:]
io1 = [input_data_1.values, output_data_1.values]

input_data_2 = dataset_2.iloc[:, :9]
output_data_2 = dataset_2.iloc[:, -2:]
io2 = [input_data_2.values, output_data_2.values]

input_data_3 = dataset_3.iloc[:, :9]
output_data_3 = dataset_3.iloc[:, -2:]
io3 = [input_data_3.values, output_data_3.values]

input_data_4 = dataset_4.iloc[:, :21]
output_data_4 = dataset_4.iloc[:, -3:]
io4 = [input_data_4.values, output_data_4.values]

input_data_5 = dataset_5.iloc[:, :13]
output_data_5 = dataset_5.iloc[:, -3:]
io5 = [input_data_5.values, output_data_5.values]

io = [io0, io1, io2, io3, io4, io5]

In [16]:
#dictionary selection by name
dict_datasets = {dataset_0.name:(dataset_0,io0),
                 dataset_1.name:(dataset_1,io1),
                 dataset_2.name:(dataset_2,io2),
                 dataset_3.name:(dataset_3,io3),
                 dataset_4.name:(dataset_4,io4),
                 dataset_5.name:(dataset_5,io4)}

In [17]:
@interact_manual
def learning_datasets(dataset_name = dict_datasets.keys(),
                  test_size = (0.1,1,0.1),
                  activation_1_layer = ['tanh', 'sigmoid'],
                  activation_2_layer = ['tanh', 'sigmoid'],
                  neurons_1_layer = (5,100,5),
                  neurons_2_layer = (5,100,5),
                  optimizer = ['adam', 'sgd'],
                  loss = ['mse'],
                  metrics = ['accuracy'],
                  epochs = (1,101,5)):
    
    head = dict_datasets[dataset_name][0].head()   #first 5 rows in dataset
    Q = dict_datasets[dataset_name][1][0].shape[0] #rows -> int
    m = dict_datasets[dataset_name][1][0].shape[1] #input columns -> int
    p = dict_datasets[dataset_name][1][1].shape[1] #output columns -> int
    
    input_data = dict_datasets[dataset_name][1][0]
    output_data = dict_datasets[dataset_name][1][1]
    
    #split data into test and training samples
    X_train, X_test, Y_train, Y_test = train_test_split(input_data, 
                                                        output_data,
                                                        test_size=test_size)
    
    print(f'{Q} sample with {m} input columns and {p} output')
    print(f'Size of train sample: {len(X_train)}')
    print(f'Size of test sample: {len(X_test)}')
    
    #create model neural network
    model = Sequential([
                    Dense(neurons_1_layer, 
                          input_dim=m,
                          activation=activation_1_layer,
                          kernel_initializer='glorot_uniform'), #xavier initialization
                    Dense(neurons_2_layer, 
                          activation=activation_2_layer,
                          kernel_initializer='glorot_uniform'),
                    Dense(p, 
                          activation='softmax', 
                          kernel_initializer='glorot_uniform'),
                    ])
    model.summary()
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=[metrics])
    
    x_axis = np.linspace(1, epochs, epochs)
    
    history = model.fit(X_train, Y_train, validation_split=0.1, epochs=epochs, verbose=0)
    error_num_val = [int(np.ceil((100*(1 - i))/(100./len(X_test)))) for i in history.history['val_accuracy']]
    
    fig1 = go.Figure()
    fig2 = go.Figure()
    fig3 = go.Figure()
    fig1.add_trace(go.Scatter(x=x_axis,
                             y=history.history['loss'],
                             name='Train sample'))
    fig1.add_trace(go.Scatter(x=x_axis,
                             y=history.history['val_loss'],
                             name='Valid sample'))
    fig2.add_trace(go.Scatter(x=x_axis,
                             y=history.history['accuracy'],
                             name='Train sample'))
    fig2.add_trace(go.Scatter(x=x_axis,
                             y=history.history['val_accuracy'],
                             name='Valid sample'))
    fig3.add_trace(go.Scatter(x=x_axis,
                             y=error_num_val,
                             name='Valid sample'))
    fig1.update_layout(title='Loss',
                       autosize=False,
                       width=800,
                       height=500,
                       yaxis=dict(title_text='loss'),
                       xaxis=dict(title_text='Epoch'))
    fig2.update_layout(title='Accuracy',
                       autosize=False,
                       width=800,
                       height=500,
                       yaxis=dict(title_text='Accuracy'),
                       xaxis=dict(title_text='Epoch'))
    fig3.update_layout(title='Errors',
                       autosize=False,
                       width=800,
                       height=500,
                       yaxis=dict(title_text='Errors'),
                       xaxis=dict(title_text='Epoch'))
    fig1.show()
    fig2.show()
    fig3.show()
        
    ev = model.evaluate(X_test, Y_test, verbose=0)
    error_num_end = int(np.ceil((100*(1 - ev[1]))/(100./len(X_test))))
    
    print(f'Accuracy test sample: {ev[1]}')
    print(f'Errors : {error_num_end} ')
    
    return head

interactive(children=(Dropdown(description='dataset_name', options=('simple', 'iris', 'cancer', 'glass', 'thyr…